Setup, Train, Test YOLOv5 models

# Setup



In [1]:
!git clone -b dev https://github.com/biggie-inc/yolov5.git  # clone repo at dev branch
%cd yolov5
!pip install -qr requirements.txt  # install dependencies (ignore errors)
# !pip install -qr requirements_2.txt # may need to specify specific TF version

Cloning into 'yolov5'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 2648 (delta 1), reused 4 (delta 1), pack-reused 2642
Receiving objects: 100% (2648/2648), 24.88 MiB | 30.62 MiB/s, done.
Resolving deltas: 100% (1734/1734), done.
/content/yolov5
     |████████████████████████████████| 276kB 9.5MB/s 


In [2]:
import cv2
import matplotlib.pyplot as plt
from collections import Counter as counter
import numpy as np

In [3]:
import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 CPU


# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [5]:
# copy the .zip file into the root directory of cloud VM
#!cp /mydrive/yolov5/yolov5_tailgate_train.zip /content/    # first train/test
# !cp /mydrive/yolov5/tailgate_handle_bounds.zip /content/  # second train/test
# !cp /mydrive/Biggie/train_test_rear_truck_photos.zip /content/ # third train/test
!cp /mydrive/Biggie/yolov5/train_test_rear_truck_photos2.zip /content/ # retraining with new bbox bounds

In [6]:
%%capture
%cd /content/
!unzip /content/train_test_rear_truck_photos2.zip
%cd ./yolov5

# Train


In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
# ONLY FOR SINGLE OBJECT DETECTION
# import glob
# import re

# txt_file_paths = glob.glob(r"../train/labels/*.txt")
# for i, file_path in enumerate(txt_file_paths):
#     # get image size
#     with open(file_path, "r") as f_o:
#         lines = f_o.readlines()

#         text_converted = []
#         for line in lines:
#             print(line)
#             numbers = re.findall("[0-9.]+", line)
#             print(numbers)
#             if numbers:

#                 # Define coordinates
#                 text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
#                 text_converted.append(text)
#                 print(i, file_path)
#                 print(text)
#         # Write file
#         with open(file_path, 'w') as fp:
#             for item in text_converted:
#                 fp.writelines("%s\n" % item)
                
#### FUTURE UPDATE: WRITE CODE THAT BYPASSES CLASSES.TXT ##### 

Start training from pretrained `--weights yolov5s.pt`, or from randomly initialized `--weights ''`. Pretrained weights are auto-downloaded from [Google Drive](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J).

**All training results are saved to `runs/exp0`** for the first experiment, then `runs/exp1`, `runs/exp2` etc. for subsequent experiments.


In [ ]:
# Start tensorboard (optional)
%reload_ext tensorboard
%tensorboard --logdir runs

In [11]:
%cd ../yolov5

/content/yolov5


In [13]:
# Train YOLOv5s on tailgate data
!python train.py --batch 64 --epochs 500 --data tg_handle_plate_photo_train.yaml --cfg yolov5s_tg_handle_plate.yaml --weights /mydrive/Biggie/yolov5/exp0_tailgate_handle/weights/last_tailgate_handle.pt --name rear_truck_photos_enlarged_bbox --nosave --cache --adam --logdir /mydrive/yolov5

Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

Namespace(adam=True, batch_size=64, bucket='', cache_images=True, cfg='./models/yolov5s_tg_handle_plate.yaml', data='./data/tg_handle_plate_photo_train.yaml', device='', epochs=500, evolve=False, global_rank=-1, hyp='data/hyp.finetune.yaml', img_size=[640, 640], local_rank=-1, logdir='/mydrive/yolov5', multi_scale=False, name='rear_truck_photos_enlarged_bbox', noautoanchor=False, nosave=True, notest=False, rect=False, resume=False, single_cls=False, sync_bn=False, total_batch_size=64, weights='/mydrive/Biggie/yolov5/exp0_tailgate_handle/weights/last_tailgate_handle.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir /mydrive/yolov5", view at http://localhost:6006/
2020-12-21 22:10:58.561077: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Hyperparameters {'lr0': 0.01, 'momentum': 0.94, 'weight_decay': 0.0005, 'giou

Removing 4th channel of pngs as needed for training

(model doesn't recogize their sRGB profile)



In [7]:
#Removing the 4th channel for the pngs as needed
#from train_test_rear_truck_photos2.zip

from glob import glob
import os
from PIL import Image
import cv2

training_pngs = sorted(glob('/content/rear_truck_photos/training_photos/*.png'))

for png in training_pngs:
    filepath, filename = os.path.split(png)
    filename_no_ext = (filename.split('.')[0])#.replace(' ', '_') # used to remove spaces
    img = Image.open(png)
    rgb_img = img.convert('RGB')
    rgb_img.save(f'{filepath}/{filename_no_ext}.jpg')


# matching txt file names to png file names - doesn't seem to work 100%
# training_txts = sorted(glob('/content/rear_truck_photos/training_photos/*.txt'))

# for txt in training_txts:
#     filepath, filename = os.path.split(txt)
#     filename_no_ext = (filename.split('.')[0]).replace(' ', '_')
#     os.rename(txt, f'{filepath}/{filename_no_ext}.txt')

# for png in training_pngs:
#     filepath, filename = os.path.split(png)
#     filename_no_ext = (filename.split('.')[0]).replace(' ', '_')
#     img = cv2.imread(png)
#     rgb_img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
#     cv2.imwrite(f'{filepath}/{filename_no_ext}.png', rgb_img)
#above option did not fix artifact issues

In [8]:
!mkdir ../3000e_training_set2
!mv ../rear_truck_photos/training_photos/*.jpg ../3000e_training_set2
!mv ../rear_truck_photos/training_photos/*.txt ../3000e_training_set2

In [9]:
!ls

data				    requirements_2.txt
detect_photo_crops.py		    requirements.txt
detect_photo_dims_to_csv.py	    tailgate_utils.py
detect_photo.py			    test.py
detect.py			    top100_csv_processing.ipynb
Dockerfile			    Top100List_BodyStyle_AllStyles.csv
hubconf.py			    train.py
inference			    tutorial.ipynb
LICENSE				    utils
models				    weights
randomize_truck_photo_train_set.py  YOLOv5_Tailgate_Detect.ipynb
README.md


# Visualize

View `runs/exp0/train*.jpg` images to see training images, labels and augmentation effects. A **Mosaic Dataloader** is used for training (shown below), a new concept developed by Ultralytics and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
Image(filename='/mydrive/yolov5/exp1_tailgate_handle/train_batch1.jpg', width=900)  # view augmented training mosaics

View `test_batch0_gt.jpg` to see test batch 0 *ground truth* labels.

In [ ]:
Image(filename='/mydrive/yolov5/exp1_tailgate_handle/test_batch0_gt.jpg', width=900)  # view test image labels

View `test_batch0_pred.jpg` to see test batch 0 *predictions*.

In [ ]:
# for some reaons the bboxes aren't recorded here, but they are when using detect.py
Image(filename='/mydrive/yolov5/exp1_tailgate_handle/test_batch0_pred.jpg', width=900)  # view test image predictions

In [ ]:
Image(filename='/mydrive/yolov5/exp14_rear_truck_photos_enlarged_bbox/results.png', width=900)

TypeError: ignored

Training losses and performance metrics are saved to Tensorboard and also to a `runs/exp0/results.txt` logfile. `results.txt` is plotted as `results.png` after training completes. Partially completed `results.txt` files can be plotted with `from utils.general import plot_results; plot_results()`.

# Detection and Testing


In [ ]:
# # import and unzip test set
# !cp /mydrive/yolov5/tailgate_.zip ../

# %cd ../
# !mkdir ./test_set/
# !mkdir ./test_set/model_output/
# # unzipping file into the test_set folder
# %cd ./test_set
# !unzip ../tailgate_test_set.zip
# # getting back to yolo folder 
# %cd ../yolov5

In [8]:
%ls

data/                               requirements_2.txt*
detect_photo_crops.py               requirements.txt*
detect_photo_dims_to_csv.py         scratchpad.py
detect_photo.py                     tailgate_utils.py
detect.py                           test.py
Dockerfile                          top100_csv_processing.ipynb
hubconf.py                          Top100List_BodyStyle_AllStyles.csv
inference/                          train.py
LICENSE                             tutorial.ipynb
models/                             utils/
randomize_truck_photo_train_set.py  weights/
README.md*                          YOLOv5_Tailgate_Detect.ipynb


In [9]:
# Single Detection

# tailgate not found process
# !python detect_photo_crops.py --weights /mydrive/Biggie/yolov5/exp8_rear_truck_photos_enlarged_bbox/weights/last_rear_truck_photos_enlarged_bbox.pt --source /content/rear_truck_photos/10288_2015FordF-150_2400.png

# proof main process works 
# !python detect_photo_crops.py --weights /mydrive/Biggie/yolov5/exp8_rear_truck_photos_enlarged_bbox/weights/last_rear_truck_photos_enlarged_bbox.pt --source /content/rear_truck_photos/10007_2015GMCSierra\ 1500_2400.png

# proof border works  
# !python detect_photo_crops.py --weights /mydrive/Biggie/yolov5/exp8_rear_truck_photos_enlarged_bbox/weights/last_rear_truck_photos_enlarged_bbox.pt --source /content/rear_truck_photos/1090_2002DodgeDakota_1280.jpg

# tailgate not found on Red Nissan Frontier
# !python detect_photo_crops.py --weights /mydrive/Biggie/yolov5/exp8_rear_truck_photos_enlarged_bbox/weights/last_rear_truck_photos_enlarged_bbox.pt --source /content/rear_truck_photos/10273_2015NissanFrontier_2400.png
 
# darken process and corner process works
# !python detect_photo_crops.py --weights /mydrive/Biggie/yolov5/exp8_rear_truck_photos_enlarged_bbox/weights/last_rear_truck_photos_enlarged_bbox.pt --source /content/rear_truck_photos/1872_2004FordRanger_1280.jpg

# another white truck works
# !python detect_photo_crops.py --weights /mydrive/Biggie/yolov5/exp8_rear_truck_photos_enlarged_bbox/weights/last_rear_truck_photos_enlarged_bbox.pt --source /content/rear_truck_photos/6698_2010GMCCanyon_2400.png

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', img_size=640, iou_thres=0.5, output='inference/output', save_txt=False, source='/content/rear_truck_photos/10007_2015GMCSierra 1500_2400.png', update=False, view_img=False, weights=['/mydrive/Biggie/yolov5/exp8_rear_truck_photos_enlarged_bbox/weights/last_rear_truck_photos_enlarged_bbox.pt'])
Using CPU

Fusing layers... 
Model Summary: 140 layers, 7.25191e+06 parameters, 6.61683e+06 gradients
image 1/1 /content/rear_truck_photos/10007_2015GMCSierra 1500_2400.png: 512x640 1 tailgates, 1 handles, 1 plates, Done. (0.409s)
Results saved to inference/output
Done. (1.082s)


Predict on Entire Folder

In [57]:
%mkdir /content/test_group_photos/

In [64]:
from glob import glob

training_pngs = sorted(glob('/content/rear_truck_photos/*.png'))[:10]

for png in training_pngs:
    deal_with_space = png.replace(' ', '\ ')
    !mv {deal_with_space} /content/test_group_photos/

In [61]:
%cd yolov5/

/content/yolov5


In [65]:
!python detect_photo_crops.py --weights /mydrive/Biggie/yolov5/exp8_rear_truck_photos_enlarged_bbox/weights/last_rear_truck_photos_enlarged_bbox.pt --source /content/test_group_photos/

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', img_size=640, iou_thres=0.5, output='inference/output', save_txt=False, source='/content/test_group_photos/', update=False, view_img=False, weights=['/mydrive/Biggie/yolov5/exp8_rear_truck_photos_enlarged_bbox/weights/last_rear_truck_photos_enlarged_bbox.pt'])
Using CPU

Fusing layers... 
Model Summary: 140 layers, 7.25191e+06 parameters, 6.61683e+06 gradients
image 1/13 /content/test_group_photos/10007_2015GMCSierra_1500_2400.png: 512x640 1 tailgates, 1 handles, 1 plates, Done. (0.414s)
image 2/13 /content/test_group_photos/10013_2015RAM2500_2400.png: 512x640 1 tailgates, 1 handles, 1 plates, Done. (0.384s)
image 3/13 /content/test_group_photos/10058_2015ChevroletSilverado 1500_2400.png: 512x640 1 tailgates, 1 handles, 1 plates, Done. (0.385s)
image 4/13 /content/test_group_photos/10063_2015RAM1500_2400.png: 512x640 1 tailgates, 1 handles, 1 plates, Done. (0.377s)
image 5/13 /content/test_group_photo